## Context2Canvas - AI ggplot2 Code Generation

Brief Elevator Pitch of our project 

## Setup the functions for prompting

In [1]:
from openai import OpenAI
client = OpenAI()

def prompt_model(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role": "user", 'content': prompt}
        ]
    )
    return completion.choices[0].message.content

## Chunk data and generate indices (setting up vector database)

User queries will be matched to indexes that best approximate the text chunks used to summarize an answer. For this assignment, you may chunk the text and then prompt the model to generate questions that are answerable by the text. The generated questions can then be used as the "documents" stored in the vector database.

In [8]:
#Load the data

import json
with open('annotations.json', 'r') as f:
    data = json.load(f)



In [9]:
#import langchain chunking library
from langchain_text_splitters import CharacterTextSplitter

#from langchain_text_splitters import RecursiveCharacterTextSplitter 
#we can use better parsing techniques in a future update


In [10]:
#set up text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)


In [ ]:
#chunk data
ids = [record["image_index"] for record in data]
#metadata = [record["general_plot_data"] for record in data] ### broken for now
types = [record["type"] for record in data]


## Build the vector database

When building the vector database, be sure to maintain a mapping between the generated questions and the chunks that can be used later to retrieve the chunks from the most similar indices to the user query provided.

You may also add the function to query the vector database that you will use later.

In [14]:
#setting up the collection
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="re-searcher")



In [ ]:
#add data to collection
collection.add(
    documents = annotations,
    metadatas = types,
    ids = ids
)

Insert of existing embedding ID: id0
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2
Insert of existing embedding ID: id3
Insert of existing embedding ID: id4
Insert of existing embedding ID: id5
Insert of existing embedding ID: id6
Insert of existing embedding ID: id7
Insert of existing embedding ID: id8
Insert of existing embedding ID: id9
Insert of existing embedding ID: id10
Insert of existing embedding ID: id11
Insert of existing embedding ID: id12
Insert of existing embedding ID: id13
Insert of existing embedding ID: id14
Insert of existing embedding ID: id15
Insert of existing embedding ID: id16
Insert of existing embedding ID: id17
Insert of existing embedding ID: id18
Insert of existing embedding ID: id19
Insert of existing embedding ID: id20
Insert of existing embedding ID: id21
Insert of existing embedding ID: id22
Insert of existing embedding ID: id23
Insert of existing embedding ID: id24
Insert of existing embedding ID: id25
Insert of existing emb

[['id984', 'id1608', 'id1547', 'id1555']]


## Analyze Input Data

In [ ]:
input_data = #data inserted by user

# what are the variables associated with this data?
prompt_model("What are the variables associated with this dataset? Return them as a python dictionary, in the format of 'variable_name: data_type'. Do not return any dialogue besides the python dictionary.")

#generate summary statistics
prompt_model(f"using {input_data}, generate me summary statistics of each variable according to these {standards}")

# Create Interesting data questions based on the variables + summary statistics
prompt_model(f"using this process for creating data questions, create an interesting data question based on the variable types and summary statistics")

# potential idea: ask user to for manual input of data question
# Provide option for the model to select a data question on its own

# gather visualization type from data question
prompt_model(f"what is the best visualization class we should use to characterize this problem?")


## Use VectorDB to query similar examples for few-shot learning

In [ ]:
results = collection.query(
    query_texts= node['query'], # Chroma will embed this for you
    n_results=3 # how many results to return
)


## Generate the graph's code based on the collected examples


In [38]:
from bert_score import BERTScorer

scorer = BERTScorer(model_type='bert-base-uncased')